# Example 5.3

We continue with the setting of the previous example. Now let $a(u, v):=\int_{\Omega} 6 \nabla u \nabla v \mathrm{d} x$ and $f(v):=\int_{\Omega} 16 v \mathrm{d} x .$ 

We consider the discrete problem:
Find $u_{h} \in V_{h}$ so that $a\left(u_{h}, v_{h}\right)=f\left(v_{h}\right)$ for all $v_{h} \in V_{h}$

(c) Compute the element matrix $A_{T}$ and the element vector $f_{T}$ for $T_{2} .$ Note, that the basis functions on one particular element are given by the composition of the basis functions on the reference element and the corresponding mapping (Lagrange Finite elements are equivalent!). This means for $\varphi_{m}:=\hat{\varphi}_{m} \circ F_{2}^{-1}$ with $m \in\{0,1,2,3\}$ compute $\left(A_{T}\right)_{m n}:=\int_{T_{2}} 6 \nabla \varphi_{n} \nabla \varphi_{m} \mathrm{d} x$ and $\left(f_{T}\right)_{m}:=\int_{T_{2}} 16 \varphi_{m} \mathrm{d} x, m, n \in\{0,1,2,3,\}$
Document the calculation process.

(d) Give the connectivity matrix $C_{T}$ for element $T_{2}$

(e) In this concrete example it turns out that all element matrices are the same. Set
up the system matrix $A$ and the vector $b$. You can also do this with NGSolve using the prepared file assemble_quads.py where the element matrices and vectors are computed. Note, that you do not have to calculate the connectivity matrices for that.

Finally we want to solve the problem, but want to impose Dirichlet boundary conditions on the whole boundary. Find $u_{h} \in V_{h, D}:=\left\{u_{h} \in V_{h}\left|u_{h}\right|_{\partial \Omega}=1\right\}$ so that $a\left(u_{h}, v_{h}\right)=f\left(v_{h}\right)$
for all $v_{h} \in V_{h, 0}=\left\{u_{h} \in V_{h}\left|u_{h}\right|_{\partial \Omega}=0\right\}$

(f) Solve the (discrete) inhomogeneous Dirichlet problem and make a sketch of the solution $u_{h}(x, y)$

---

Hints:
. Note that in the sketch the local numbering of degrees of freedom (basis function numbering, functional numbering) is indicated inside the element while the global numbering is put on the vertices of the triangulation. Thus for example the global functional $\psi_{4}$ equals the local functional $\psi_{T_{0}}^{1}$ and $\psi_{T_{1}}^{0}$
In assemble_quads.py the element matrices for the same problem are computed and shown. You may want to use this to verify your computations in (c).

# 5.3 (c)
Compute the element matrix $A_{T}$ and the element vector $f_{T}$ for $T_{2} .$ Note, that the basis functions on one particular element are given by the composition of the basis functions on the reference element and the corresponding mapping (Lagrange Finite elements are equivalent!). This means for $\varphi_{m}:=\hat{\varphi}_{m} \circ F_{2}^{-1}$ with $m \in\{0,1,2,3\}$ compute $\left(A_{T}\right)_{m n}:=\int_{T_{2}} 6 \nabla \varphi_{n} \nabla \varphi_{m} \mathrm{d} x$ and $\left(f_{T}\right)_{m}:=\int_{T_{2}} 16 \varphi_{m} \mathrm{d} x, m, n \in\{0,1,2,3,\}$
Document the calculation process.

In [ ]:
from sympy import *
init_session()
#x, y = symbols('x y')
M = 4
# expr = 2*(1-y)*(2*x-1)
print('#'*10+'phi'+'#'*10)
phis = []
phis.append(2*(1-y)*(2*x-1)) # phi_5
phis.append((2*y-1)*(2*x-1)) # phi_2
phis.append(2*(2*y-1)*(1-x)) # phi_6
phis.append(4*(1-y)*(1-x)) # phi_8
for m, phi in enumerate(phis):
    print(f"phi_{m}=", phi)

print('#'*10+'Grad(phi)'+'#'*10)
grad_phis = []
for phi in phis:
    dx = diff(phi, x)
    dy = diff(phi, y)
    grad_phis.append([dx, dy])
for m, grad in enumerate(grad_phis):
    print(f"grad(phi_{m})=", grad)

print('#'*10+'(A_T)_mn'+'#'*10)
A_T2 = [[0 for i in range(M)] for _ in range(M)]
#[print(row) for row in A_T2];
for m, row in enumerate(A_T2):
    for n, ele in enumerate(row):#
        prod = grad_phis[n][0]*grad_phis[m][0] + grad_phis[n][1]*grad_phis[m][1]
        A_T2[m][n] = 6*integrate(prod, (x, 0.5, 1), (y, 0.5, 1))
        if (m==0 and n==0):
            print("sample:", 6*prod)
[print(f"{row}") for row in A_T2];

print('#'*10+'(f)_n'+'#'*10)
fs = [0 for i in range(M)]
#[print(row) for row in A_T2];
for m, f in enumerate(fs):#
    fs[m] = 16*integrate(phis[m], (x, 0.5, 1), (y, 0.5, 1))
print(fs) 


# 5.3 (e)
(e) In this concrete example it turns out that all element matrices are the same. Set
up the system matrix $A$ and the vector $b$. You can also do this with NGSolve using the prepared file assemble_quads.py where the element matrices and vectors are computed. Note, that you do not have to calculate the connectivity matrices for that.

In [ ]:
import netgen.gui
from ngsolve import *
from netgen.geom2d import SplineGeometry
import matplotlib.pyplot as plt
# Taken from "assemble_quads.py"
mesh = Mesh("quads.vol.gz")
V = H1(mesh, order=1, dirichlet="bottom|right|top|left")
# u, v = V.TnT()
# bfi =  BilinearForm(V)
# bfi += 6*InnerProduct(grad(u), grad(v))*dx
# lfi = LinearForm(V)
# lfi += 16*v*dx
bfi = SymbolicBFI(6*grad(V.TrialFunction())*grad(V.TestFunction()))
lfi = SymbolicLFI(16*V.TrialFunction())

#storage to gather the global matrix:
a = Matrix(V.ndof,V.ndof)
for i in range(V.ndof):
    for j in range(V.ndof):
        a[i,j] = 0.0;

#storage to gather the global vector:
f = Vector(V.ndof)
for i in range(V.ndof):
    f[i] = 0.0

for el in V.Elements():
    print("---------------------------------------- ")
    print("currently on element: ", el.nr)
    print("the vertices of this elements are:\n", el.vertices)
    print("the degrees of freedom of this element are:\n", el.dofs)
    
    dofs = el.dofs
    elmat = bfi.CalcElementMatrix(el.GetFE(),el.GetTrafo())
    print("the element matrix of this element is:\n", elmat)
    elvec = lfi.CalcElementVector(el.GetFE(),el.GetTrafo())
    print("the element vector of this element is:\n", elvec)
    
print("the final assembled matrix is:\n",a)
print("the final assembled vector is:\n",f)

u = GridFunction(V)
u.vec[0:8]=1
u.vec[8]=2.5

print(u.vec)

Draw(u,mesh,"u")

# 5.3 (f)
Finally we want to solve the problem, but want to impose Dirichlet boundary conditions on the whole boundary.

Find $u_{h} \in V_{h, D}:=\left\{u_{h} \in V_{h}\left|u_{h}\right|_{\partial \Omega}=1\right\}$ so that $a\left(u_{h}, v_{h}\right)=f\left(v_{h}\right)$
for all $v_{h} \in V_{h, 0}=\left\{u_{h} \in V_{h}\left|u_{h}\right|_{\partial \Omega}=0\right\}$

(f) Solve the (discrete) inhomogeneous Dirichlet problem and make a sketch of the solution $u_{h}(x, y)$

In [ ]:
mesh = Mesh("quads.vol.gz")
print("Names of boundaries of domain:")
print(mesh.GetBoundaries())
V = H1(mesh, order=1, dirichlet="bottom|right|top|left") # FES... (inhom) Dirichlet BC everywhere#

# Trial and Testfunctions
u, v = V.TnT()
# Set up the Bilinear Form
a =  BilinearForm(V)
a += 6*InnerProduct(grad(u), grad(v))*dx
# Set up the Linear Form
f = LinearForm(V)
f += 16*v*dx

# Check the DoFs
freedofs = V.FreeDofs()
print("Free DoFs:")
print (freedofs)

# Set Inhomogeneous Dirichlet values via
u_D = GridFunction(V)
u_D.Set(1, BND) # u_D = 1 on whole boundary

# Assemble the BLF Matrix and the LF vector
with TaskManager(): # with Multithreading
    a.Assemble()
    f.Assemble()

# We now have to split the problem into a homogenous and inhomogenous setting: ("Homogenisation procedure --> FAM?")
# Split the solution: u=u0+uD
# (see NGSolve tutorial: <https://ngsolve.org/docu/latest/i-tutorials/unit-1.3-dirichlet/dirichlet.html>)
# and calculate the "new" right hand side: A(u_0,v) = f(v) - A(u_D,v) = r(v) for the "homogenyzed system"
r = LinearForm(V)
r.vec.data = f.vec - a.mat * u_D.vec

# Solve the homogenous problem, find u_0 s.t. ....
u_0 = GridFunction(V)
u_0.vec.data = a.mat.Inverse(V.FreeDofs(), inverse = 'sparsecholesky') * r.vec

# To get the solution of the original, inhom. system, we add the 2 parts together
# sol = u = u0+uD
sol = GridFunction(V)
sol = u_0 + u_D

## Alternative: Using BVP ("automated" version of the above)
# gfu.Set(1, BND)
# c = Preconditioner(a,"local")   #<- Jacobi preconditioner
## c = Preconditioner(a,"direct") #<- sparse direct solver
# c.Update()
# solvers.BVP(bf=a, lf=f, gf=gfu, pre=c)

Draw(sol, mesh, 'sol_u')